In [8]:
from photutils.segmentation import (SourceCatalog, deblend_sources, detect_sources, detect_threshold)
from astropy.io import fits
from photutils.aperture import CircularAperture
from astropy.stats import mad_std
import numpy as np
from astropy.visualization import ZScaleInterval
import matplotlib.pyplot as plt

In [9]:
#Abrimos el fits de las estrellas
star_1b = fits.open('Star1_B.fits')[0].data
star_1v = fits.open('Star1_V.fits')[0].data
star_2b = fits.open('Star2_B.fits')[0].data
star_2v = fits.open('Star2_V.fits')[0].data

In [10]:
fits.open('Star1_B.fits')[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 4096                                                  
NAXIS2  =                 4096                                                  
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
OBSERVER= 'Martinez, Oyarzo, Vargas' / SBIGFITSEXT The name of the observer     
ORIGIN  = 'MAS500  '           / Description of location                        
TELESCOP= 'MAS500  '           / SBIGFITSEXT The model Telescope                
FOCALLEN=                6500. / SBIGFITSEXT Telescope focal length in mm       
APTDIA  =                 500. / SBIGFITSEXT Aperture diameter in mm            
SBUUID  = '{da52125b-74f6-49

In [11]:
#Calcular desviación del fondo (background noise)
bkg_1b = mad_std(star_1b)
bkg_1v = mad_std(star_1v)
bkg_2b = mad_std(star_2b)
bkg_2v = mad_std(star_2v)

In [12]:
#Threshold, para cada estrella
threshold_1b = detect_threshold(star_1b, nsigma = 5)
threshold_1v = detect_threshold(star_1v, nsigma = 5)
threshold_2b = detect_threshold(star_2b, nsigma = 5)
threshold_2v = detect_threshold(star_2v, nsigma = 5)
#Detectamos fuentes
sour_1b = detect_sources(star_1b, threshold_1b, npixels=50)
sour_1v = detect_sources(star_1v, threshold_1v, npixels=50)
sour_2b = detect_sources(star_2b, threshold_2b, npixels=50)
sour_2v = detect_sources(star_2v, threshold_2v, npixels=50)

In [13]:
#Crear catálogos de las fuentes detectadas
cat_1b = SourceCatalog(star_1b, sour_1b)
cat_1v = SourceCatalog(star_1v, sour_1v)
cat_2b = SourceCatalog(star_2b, sour_2b)
cat_2v = SourceCatalog(star_2v, sour_2v)

In [20]:
#Lo pasamos a tabla
tabla_1b = cat_1b.to_table()
tabla_1v = cat_1v.to_table()
tabla_2b = cat_2b.to_table()
tabla_2v = cat_2v.to_table()


In [21]:
#Buscamos la fuente que es la estrella (mayor área)
star_1b_index = np.argmax(tabla_1b['area'])
star_1v_index = np.argmax(tabla_1v['area'])

star_2b_index = np.argmax(tabla_2b['area'])
star_2v_index = np.argmax(tabla_2v['area'])

# Dispersión (σ) del eje mayor y menor de la elipse ajustada
sigma_major1b = tabla_1b['semimajor_sigma'][star_1b_index].value
sigma_minor1b = tabla_1b['semiminor_sigma'][star_1b_index].value

sigma_major1v = tabla_1v['semimajor_sigma'][star_1v_index].value
sigma_minor1v = tabla_1v['semiminor_sigma'][star_1v_index].value

sigma_major2b = tabla_2b['semimajor_sigma'][star_2b_index].value
sigma_minor2b = tabla_2b['semiminor_sigma'][star_2b_index].value

sigma_major2v = tabla_2v['semimajor_sigma'][star_2v_index].value
sigma_minor2v = tabla_2v['semiminor_sigma'][star_2v_index].value

# FWHM promedio en píxeles, el 2.355 es por formula
fwhm_1b = 2.355 * np.mean([sigma_major1b, sigma_minor1b])
print(f"FWHM HIP 117678 Filtro B = {fwhm_1b} píxeles")

fwhm_1v = 2.355 * np.mean([sigma_major1v, sigma_minor1v])
print(f"FWHM HIP 117678 Filtro V = {fwhm_1v} píxeles")

fwhm_2b = 2.355 * np.mean([sigma_major2b, sigma_minor2b])
print(f"FWHM HIP 116375 Filtro B = {fwhm_2b} píxeles")

fwhm_2v = 2.355 * np.mean([sigma_major2v, sigma_minor2v])
print(f"FWHM HIP 116375 Filtro V = {fwhm_2v} píxeles")

# Conversión a arcosegundos
pixscale = 0.36  # arcsec/pixel (lo obtuve del header) PIXSCALE=0.36

seeing_1b = fwhm_1b * pixscale
seeing_1v = fwhm_1v * pixscale
seeing_2b = fwhm_2b * pixscale
seeing_2v = fwhm_2v * pixscale

print(f"\nSeeing HIP 117678 Filtro B = {seeing_1b} arcsec")
print(f"Seeing HIP 117678 Filtro V = {seeing_1v} arcsec")
print(f"Seeing HIP 116375 Filtro B = {seeing_2b} arcsec")
print(f"Seeing HIP 116375 Filtro V = {seeing_2v} arcsec")

FWHM HIP 117678 Filtro B = 13.171910827067643 píxeles
FWHM HIP 117678 Filtro V = 12.415647571216557 píxeles
FWHM HIP 116375 Filtro B = 10.734818616658771 píxeles
FWHM HIP 116375 Filtro V = 12.423916129599577 píxeles

Seeing HIP 117678 Filtro B = 4.741887897744351 arcsec
Seeing HIP 117678 Filtro V = 4.4696331256379604 arcsec
Seeing HIP 116375 Filtro B = 3.864534701997157 arcsec
Seeing HIP 116375 Filtro V = 4.472609806655847 arcsec


In [22]:
#Promedio total
valores = [seeing_1b, seeing_1v, seeing_2b, seeing_2v]
seeing_promedio = np.mean(valores)
print(f"Seeing promedio = {seeing_promedio} arcsec")

Seeing promedio = 4.387166383008829 arcsec
